In [455]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


In [456]:
person = 'Luc Rieffel'
data = pd.read_csv(f'{person}/autosleep-{person}.csv')
data['date'] = pd.to_datetime(data['ISO8601'], errors='coerce')
data['date'] = data['date'].apply(lambda x: x.tz_localize(None).normalize() if x is not pd.NaT else pd.NaT)
data['year'] = data['date'].dt.year
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.day
data['inBed_minutes'] = pd.to_timedelta(data['inBed']).dt.total_seconds() / 60
data['day_of_week'] = data['date'].dt.dayofweek
data['is_weekend'] = data['day_of_week'].apply(lambda x: 1 if x >= 5 else 0)
data['date'][len(data) - 10:]

/tmp/ipykernel_2393840/611133627.py:3: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  data['date'] = pd.to_datetime(data['ISO8601'], errors='coerce')


35   2024-03-31
36   2024-04-01
37   2024-04-02
38   2024-04-03
39   2024-04-04
40   2024-04-05
41   2024-04-06
42   2024-04-07
43   2024-04-08
44   2024-04-09
Name: date, dtype: datetime64[ns]

In [457]:
def rename_columns(df):
  column_mappings = {
      'Name': 'name',
      'Date': 'date',
      'Grogginess': 'grogginess',
      'How many alarms did you set?': 'num_alarms',
      'What time did you set your first alarm for?': 'first_alarm',
      'Did you take sleep-aiding medicine (not weed, ie melatonin/antihystamine)?': 'sleep_medicine',
      "What was the temperature when you woke up? (Don't include °F, just the number)": 'waking_temp',
      'Were you intoxicated when you went to sleep?': 'intoxicated',
      'Were you sick when you went to sleep?': 'sick',
      'Did you eat within an hour of going to bed?': 'eat_before_bed',
      'Did you sleep alone?': 'sleep_alone',
      'Did you sleep in your own bed/room?': 'own_bed',
      'How stressed were you last night?': 'stress',
      'Did you use your phone before going to sleep?': 'phone_before_bed',
      "When was the latest you ingested caffeine before going to bed? (Don't answer if N/A)": 'caffeine_before_bed'
  }
  
  df_renamed = df.rename(columns=column_mappings)
  return df_renamed

In [458]:
suppl = rename_columns(pd.read_csv('form.csv'))

suppl['date'] = pd.to_datetime(suppl['date'], format='%m/%d/%Y').dt.normalize()


def time_to_minutes(time_str):
  if pd.isna(time_str) or time_str == '':
      return 0
  time = pd.to_datetime(time_str, format='%I:%M:%S %p')
  return time.hour * 60 + time.minute

suppl['first_alarm'] = suppl['first_alarm'].apply(time_to_minutes)
suppl['caffeine_before_bed'] = suppl['caffeine_before_bed'].apply(time_to_minutes)
lag_periods = 1
suppl['grogginess_lag1'] = suppl['grogginess'].shift(lag_periods)
suppl = suppl[suppl['name'] == person].fillna(0)
# suppl

In [459]:
merged_data = pd.merge(suppl, data, on='date', how='inner').fillna(0)

merged_data.head(10)


,Timestamp,name,date,grogginess,num_alarms,first_alarm,sleep_medicine,waking_temp,intoxicated,sick,...,respMin,respMax,tags,notes,year,month,day,inBed_minutes,day_of_week,is_weekend
0,2/20/2024 8:32:10,Luc Rieffel,2024-02-20,3,1,580,1,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,20,540.000000,1,0
1,2/21/2024 11:25:14,Luc Rieffel,2024-02-21,5,3,580,1,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,21,539.000000,2,0
2,2/22/2024 11:05:02,Luc Rieffel,2024-02-22,8,3,580,1,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,22,576.000000,3,0
3,2/23/2024 11:03:56,Luc Rieffel,2024-02-23,7,0,0,1,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,23,448.000000,4,0
4,2/24/2024 13:19:27,Luc Rieffel,2024-02-24,8,1,660,1,67,0,0,...,0.0,0.0,0.0,0.0,2024,2,24,384.000000,5,1
5,2/25/2024 13:37:49,Luc Rieffel,2024-02-25,4,0,0,0,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,25,614.000000,6,1
6,2/26/2024 11:05:02,Luc Rieffel,2024-02-26,4,3,580,1,67,0,0,...,0.0,0.0,0.0,0.0,2024,2,26,328.000000,0,0
7,2/27/2024 11:07:32,Luc Rieffel,2024-02-27,1,3,580,1,68,0,0,...,0.0,0.0,0.0,0.0,2024,2,27,480.000000,1,0
8,2/28/2024 11:15:24,Luc Rieffel,2024-02-28,3,3,580,1,66,0,0,...,0.0,0.0,0.0,0.0,2024,2,28,507.000000,2,0
9,2/29/2024 11:38:59,Luc Rieffel,2024-02-29,4,3,580,0,67,0,0,...,0.0,0.0,0.0,0.0,2024,2,29,710.983333,3,0


In [460]:
features = ['sleepBPM', 'sleepBPMAvg7', 'dayBPM', 'dayBPMAvg7', 'wakingBPM', 'wakingBPMAvg7', 
            'hrv', 'hrvAvg7', 'sleepHRV', 'sleepHRVAvg7', 'SpO2Avg', 'SpO2Min', 'SpO2Max', 
            # 'respAvg', 'respMin', 'respMax', 
            #'year',
             'month', 
            #'day', 
            'inBed_minutes', 'day_of_week', 'is_weekend', ]
features += ['first_alarm', 'num_alarms', 'sleep_medicine', 'waking_temp', 'intoxicated', 'sick', 'eat_before_bed', 'sleep_alone', 'own_bed', 'stress', 'phone_before_bed', 'caffeine_before_bed', 'grogginess_lag1']

target = ['grogginess']
# merged_data[features].head().to_csv('out.csv')
# from sklearn.model_selection import train_test_split

X = merged_data[features]  # Features
y = merged_data[target]  # Assuming you have a target variable

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# split_point = int(len(X) * 0.9)
split_point = len(X) - 1
X_train, X_test = X[:split_point], X[split_point:]
y_train, y_test = y[:split_point], y[split_point:]

X_test.head()

,sleepBPM,sleepBPMAvg7,dayBPM,dayBPMAvg7,wakingBPM,wakingBPMAvg7,hrv,hrvAvg7,sleepHRV,sleepHRVAvg7,...,waking_temp,intoxicated,sick,eat_before_bed,sleep_alone,own_bed,stress,phone_before_bed,caffeine_before_bed,grogginess_lag1
38,61.3,63.4,0.0,0.0,55.0,60.7,184,107,100,65,...,66,0,0,0,1,1,4,0,0,5.0


In [461]:
from torch.utils.data import TensorDataset, DataLoader

np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(42)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# torch.cuda.set_device(torch.cuda.current_device())

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)
batch_size = 32  # Adjust based on your computational resources

training_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(dataset=training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


In [462]:
class GrogginessModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(GrogginessModel, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size) 
        self.layer2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.layer2(x)
        return x

input_size = len(features) 
hidden_size = 5  
output_size = 1
model = GrogginessModel(input_size, hidden_size, output_size)
len(features)

30

In [463]:
criterion = nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=0.001) 
# optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [464]:
def evaluate_model(model, test_loader, criterion):
  model.eval()  # Set the model to evaluation mode
  total_loss = 0.0
  with torch.no_grad():  # No need to track gradients
    for inputs, targets in test_loader:
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      total_loss += loss.item() * inputs.size(0)
  avg_loss = total_loss / len(test_loader.dataset)
  return avg_loss
  
class EarlyStopping:
  def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
    self.patience = patience
    self.verbose = verbose
    self.counter = 0
    self.best_score = None
    self.early_stop = False
    self.val_loss_min = np.Inf
    self.delta = delta
    self.path = path

  def __call__(self, val_loss, model):
    score = -val_loss

    if self.best_score is None:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
    elif score < self.best_score + self.delta:
      self.counter += 1
      # print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
      if self.counter >= self.patience:
          self.early_stop = True
    else:
      self.best_score = score
      self.save_checkpoint(val_loss, model)
      self.counter = 0

  def save_checkpoint(self, val_loss, model):
    if self.verbose:
      print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
    torch.save(model.state_dict(), self.path)
    self.val_loss_min = val_loss


In [465]:
epochs = 2000
early_stopping = EarlyStopping(patience=1000, verbose=True)

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    val_loss = evaluate_model(model, test_loader, criterion)
    early_stopping(val_loss, model)
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Optimize
        
        running_loss += loss.item() * inputs.size(0)  # Total loss for the batch

    epoch_loss = running_loss / len(train_loader.dataset)  # Average loss for the epoch

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f}')


Validation loss decreased (inf --> 225.735565).  Saving model ...
Epoch 1/2000, Loss: 338.1221
Validation loss decreased (225.735565 --> 143.789398).  Saving model ...
Validation loss decreased (143.789398 --> 77.958961).  Saving model ...
Validation loss decreased (77.958961 --> 32.144596).  Saving model ...
Validation loss decreased (32.144596 --> 6.584228).  Saving model ...
Validation loss decreased (6.584228 --> 0.136052).  Saving model ...


Validation loss decreased (0.136052 --> 0.097133).  Saving model ...
Validation loss decreased (0.097133 --> 0.033774).  Saving model ...
Validation loss decreased (0.033774 --> 0.001717).  Saving model ...
Validation loss decreased (0.001717 --> 0.000348).  Saving model ...
Validation loss decreased (0.000348 --> 0.000030).  Saving model ...
Epoch 1001/2000, Loss: 2.3286
Validation loss decreased (0.000030 --> 0.000007).  Saving model ...
Validation loss decreased (0.000007 --> 0.000003).  Saving model ...


In [466]:
print(features)

['sleepBPM', 'sleepBPMAvg7', 'dayBPM', 'dayBPMAvg7', 'wakingBPM', 'wakingBPMAvg7', 'hrv', 'hrvAvg7', 'sleepHRV', 'sleepHRVAvg7', 'SpO2Avg', 'SpO2Min', 'SpO2Max', 'month', 'inBed_minutes', 'day_of_week', 'is_weekend', 'first_alarm', 'num_alarms', 'sleep_medicine', 'waking_temp', 'intoxicated', 'sick', 'eat_before_bed', 'sleep_alone', 'own_bed', 'stress', 'phone_before_bed', 'caffeine_before_bed', 'grogginess_lag1']


In [467]:
# realtime_data_values = X_test[features].iloc[0]
# realtime_data_values['wakingBPM'] = 100
# print(realtime_data_values)
# realtime_data_values = realtime_data_values.tolist()


# from datetime import date
# # year, month, day = [int(x) for x in X_test[['year', 'month', 'day']].iloc[0].tolist()]


# date_of_test = str(date(year, month, day))
# # print(date_of_test)
# original_grog = float(suppl[suppl['date'] == date_of_test]['grogginess'])
# print(original_grog)

In [468]:

# realtime_data_tensor = torch.tensor([realtime_data_values], dtype=torch.float32)  # Wrap in a list to keep dimensions

# model.eval()  # Set the model to evaluation mode
# with torch.no_grad():
#   predicted_grogginess = model(realtime_data_tensor)
# print(predicted_grogginess[0])
# print(original_grog)


In [469]:

avg_test_loss = evaluate_model(model, test_loader, criterion)
print(f'Average test loss: {avg_test_loss:.4f}')

Average test loss: 0.3402


In [470]:
model = GrogginessModel(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('checkpoint.pt'))
model.eval()

total_loss = 0.0
for inputs, targets in test_loader:
  with torch.no_grad():
    outputs = model(inputs)
    loss = criterion(outputs, targets)
    total_loss += loss.item() * inputs.size(0)

avg_loss = total_loss / len(test_loader.dataset)
print(f'Average test loss: {avg_loss:.4f}')

Average test loss: 0.0000


In [471]:
# realtime_data_values = X_test[features].iloc[0]
# # realtime_data_values['wakingBPM'] = 100
# # print(realtime_data_values)
# realtime_data_values = realtime_data_values.tolist()


# from datetime import date
# year, month, day = [int(x) for x in X_test[['year', 'month', 'day']].iloc[0].tolist()]

# date_of_test = str(date(year, month, day))
# # print(date_of_test)
# original_grog = float(suppl[suppl['date'] == date_of_test]['grogginess'])
# print(original_grog)


# realtime_data_tensor = torch.tensor([realtime_data_values], dtype=torch.float32)  # Wrap in a list to keep dimensions

# model.eval()  # Set the model to evaluation mode
# with torch.no_grad():
#   predicted_grogginess = model(realtime_data_tensor)
# print(predicted_grogginess[0])
# print(original_grog)
